
## 1. INTRODUCTION

Bulldozers are used in the construction sector for a variety of tasks including land clearing, dozing, handling scrap, and hauling other large construction equipment. In order to improve the quality of work, complete specific tasks with less human interference, and manage complex projects, a significant rate of adoption of technologically advanced equipment has been seen in the construction industry and this is expected to increase demand for dozers in the construction industry **[Reference](https://www.bulldozer.in/blog/bulldozing-market-future-scope-and-growth-factors/)**.

In this notebook, we will try to predict the sale price of bulldozer based on its usage, equipment type, and configuration.

<h4 style="background-color:#0b0504;color:white;border-radius: 8px; padding:12px">1.1. PROBLEM DEFINITION/GOAL</h4>

* Given that we are attempting to predict a number, the problem we are exploring is a regression problem.<br>
* Our goal is to forecast the future sale price of a bulldozer based on its features as well as the prices of similar bulldozers that have previously been sold.

<h4 style="background-color:#0b0504;color:white;border-radius: 8px; padding:12px">1.2. DATA SOURCE</h4>

The data was obtained from the **[Kaggle](https://www.kaggle.com/competitions/bluebook-for-bulldozers/data)** Bluebook for Bulldozers competition and it contains 3 main datasets:
* **Train.csv** (the training set), which contains data through the end of 2011. It includes thes `SalePrice`, which is the target variable.
* **Valid.csv** (the validation set), which contains data from January 1, 2012 - April 30, 2012. 
* **Test.csv** (the test set), which contains data from May 1, 2012 - November 2012. It does not contain the `SalePrice` attribute, since this is the what we'll try to predict.

<h4 style="background-color:#0b0504;color:white;border-radius: 8px; padding:12px">1.3. EVALUATION</h4>

The evaluation benchmark for this project is the [`RMSLE`](https://www.kaggle.com/competitions/bluebook-for-bulldozers/overview/evaluation) (root mean squared log error) between the actual and predicted auction prices. Our goal is to get the RMSLE value as low as possible.

<h4 style="background-color:#0b0504;color:white;border-radius: 8px; padding:12px">1.4. FEATURES</h4>

First Ten Features:
* **SalesID:** Unique identifier of a particular sale of a machine at auction
* **MachineID:** Identifier for a particular machine;  machines may have multiple sales
* **ModelID:** Identifier for a unique machine model (i.e. fiModelDesc)
* **Datasource:** Source of the sale record;  some sources are more diligent about reporting attributes of the machine than others.  Note that a particular datasource may report on multiple auctioneerIDs.
* **auctioneerID:** Identifier of a particular auctioneer, i.e. company that sold the machine at auction.  Not the same as datasource.
* **YearMade:** Year of manufacturer of the Machine
* **MachineHoursCurrentMeter:** Current usage of the machine in hours at time of sale (saledate);  null or 0 means no hours have been reported for that sale
* **UsageBand:** Value (low, medium, high) calculated comparing this particular Machine-Sale hours to average usage for the fiBaseModel;  e.g. 'Low' means this machine has less hours given it's lifespan relative to average of fiBaseModel.
* **Saledate:** Time of sale
* **Saleprice:** Cost of sale in USD

View complete Data Dictionary **[HERE](https://docs.google.com/spreadsheets/d/1vslmSkFUnyNdC177hA1MfIMk5gd_hZbEM1-OBlpubNY/edit?usp=sharing)**.



## 1. Importing library and Loading Data

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">1.1. LOADING THE TOOLS & DATA</h4>

In [1]:
import pandas as pd
train_path="Train.csv"
test_path="Test.csv"
valid_path="Valid.csv"
df_path="TrainAndValid.csv"
validsoln_path="ValidSolution.csv"
train_data=pd.read_csv(train_path, low_memory=False, parse_dates=['saledate'])
test_data=pd.read_csv(test_path,parse_dates=['saledate'])
valid_data=pd.read_csv(valid_path,parse_dates=['saledate'])
df_data=pd.read_csv(df_path,low_memory=False,parse_dates=['saledate'])
soln_data=pd.read_csv(validsoln_path)
#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">1.2 DATA ANALYSIS</h4>

In [2]:
train_data.shape,test_data.shape,df_data.shape

((401125, 53), (12457, 52), (412698, 53))

In [3]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412698 entries, 0 to 412697
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   412698 non-null  int64         
 1   SalePrice                 412698 non-null  float64       
 2   MachineID                 412698 non-null  int64         
 3   ModelID                   412698 non-null  int64         
 4   datasource                412698 non-null  int64         
 5   auctioneerID              392562 non-null  float64       
 6   YearMade                  412698 non-null  int64         
 7   MachineHoursCurrentMeter  147504 non-null  float64       
 8   UsageBand                 73670 non-null   object        
 9   saledate                  412698 non-null  datetime64[ns]
 10  fiModelDesc               412698 non-null  object        
 11  fiBaseModel               412698 non-null  object        
 12  fi

In [4]:
df_data.sort_values(by=['saledate'], inplace=True, ascending=True)

In [9]:
train_df=df_data.copy()

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">1.3 FEATURES ENGINNERING</h4>

In [10]:
#feture enginnering
train_df["saleyear"] = train_df['saledate'].dt.year
train_df["salemonth"] = train_df['saledate'].dt.month
train_df["saleday"] = train_df['saledate'].dt.day
new_df=train_df.drop('saledate',axis=1)

In [12]:
obj_cols = new_df.select_dtypes(include='object').columns
new_df[obj_cols] = new_df[obj_cols].astype('category')


In [13]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 205615 to 409203
Data columns (total 55 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   SalesID                   412698 non-null  int64   
 1   SalePrice                 412698 non-null  float64 
 2   MachineID                 412698 non-null  int64   
 3   ModelID                   412698 non-null  int64   
 4   datasource                412698 non-null  int64   
 5   auctioneerID              392562 non-null  float64 
 6   YearMade                  412698 non-null  int64   
 7   MachineHoursCurrentMeter  147504 non-null  float64 
 8   UsageBand                 73670 non-null   category
 9   fiModelDesc               412698 non-null  category
 10  fiBaseModel               412698 non-null  category
 11  fiSecondaryDesc           271971 non-null  category
 12  fiModelSeries             58667 non-null   category
 13  fiModelDescriptor       

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">2.1.HANDLING MISSING VALUES</h4>

In [15]:
#checking the numeric datatype
num_cols=new_df.select_dtypes(include=['int64','float64'],exclude=[ "datetime64[ns]"]).columns.tolist()
# numeric_cols = df.select_dtypes(include=np.number, exclude=["id", "date"]).columns.tolist()
missing_num_cols=new_df[num_cols].isna().any()
missing_cols=missing_num_cols[missing_num_cols==True].index.tolist()
missing_cols

['auctioneerID', 'MachineHoursCurrentMeter']

In [16]:
#auctioneerID & MachineHoursCurrentMeter has missing values so 
new_df[missing_cols]=new_df[missing_cols].fillna(df_data[missing_cols].median())

In [17]:
for label, content in new_df.items():
    if not pd.api.types.is_numeric_dtype(content):
        # Turn categories to numbers +1
        new_df[label] = pd.Categorical(content).codes+1

In [18]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412698 entries, 205615 to 409203
Data columns (total 55 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   412698 non-null  int64  
 1   SalePrice                 412698 non-null  float64
 2   MachineID                 412698 non-null  int64  
 3   ModelID                   412698 non-null  int64  
 4   datasource                412698 non-null  int64  
 5   auctioneerID              412698 non-null  float64
 6   YearMade                  412698 non-null  int64  
 7   MachineHoursCurrentMeter  412698 non-null  float64
 8   UsageBand                 412698 non-null  int8   
 9   fiModelDesc               412698 non-null  int16  
 10  fiBaseModel               412698 non-null  int16  
 11  fiSecondaryDesc           412698 non-null  int16  
 12  fiModelSeries             412698 non-null  int8   
 13  fiModelDescriptor         412698 non-nu

***

### Data Visualization and Outliers Detection

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">3.1. Outliers Detection Using Boxplot</h4>

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_box(data):
    
#     plt.figure(figsize=(30,80))
#     variables = data.columns

#     rows = 25
#     cols = 4
    
#     for i, variable in enumerate(variables):
#         plt.subplot(rows, cols, i+1)
#         sns.boxplot(x=data[variable])
#         plt.xticks(rotation=90, fontsize=10)
#         plt.xlabel(variable, fontsize=12)

#     # Display the plot
#     plt.subplots_adjust(wspace=0.3, hspace=0.9)

#     plt.show()


In [ ]:
# plot_box(new_df)

***

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">3.2. Outliers Removing</h4>

In [19]:
import numpy as np
def remove_outliers(df,cols):
    data_clean=df.copy()
    for col in cols:
        if df[col].dtype != 'datetime64':
            Q1=data_clean[col].quantile(0.25)
            Q3=data_clean[col].quantile(0.70)
            IQR=Q3-Q1

            #calculatiog upper and lower bound
            lower_bound=Q1-1.5*IQR
            upper_bound=Q3+1.5*IQR

            data_clean[col]=np.where(data_clean[col]>upper_bound,upper_bound,np.where (data_clean[col]<lower_bound,lower_bound,data_clean[col]))

    return data_clean

In [20]:
df=remove_outliers(new_df,new_df.columns)

In [21]:
df.shape

(412698, 55)

In [22]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleyear,salemonth,saleday
205615,1646770.0,9500.0,1126363.0,8434.0,132.0,6.0,1974.0,0.0,0.0,4572.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1989.0,1.0,17.0
274835,1821514.0,14000.0,1194089.0,10150.0,132.0,6.0,1980.0,0.0,0.0,1820.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1989.0,1.0,31.0
141296,1505138.0,50000.0,1473654.0,4139.0,132.0,6.0,1978.0,0.0,0.0,2348.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1989.0,1.0,31.0
212552,1671174.0,16000.0,1327630.0,8591.0,132.0,6.0,1980.0,0.0,0.0,1819.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1989.0,1.0,31.0
62755,1329056.0,22000.0,1336053.0,4089.0,132.0,6.0,1984.0,0.0,0.0,2119.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1989.0,1.0,31.0


***

### Modeling:

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">4.1. Importing Libraries</h4>

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,accuracy_score, mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [24]:
y=df['SalePrice']
X=df.drop('SalePrice',axis=1)

In [ ]:
model=RandomForestRegressor(n_jobs=-1)
model.fit(X,y)
model.score(X,y)

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px"> Random Forest Yields the 98% Accuracy</h4>

***

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">4.2. Spliting the data in train and valid set</h4>

In [25]:
#validation set as described in the original problem from January 1, 2012 - April 30, 2012.
df_data=df.copy()
valid_set = df_data[df_data.saleyear==2012]

#anything other than the data of theyear 2012 falls in the training set.

train_set = df_data[df_data.saleyear!= 2012]


In [26]:
# Split data into X and y
X_train , y_train = train_set.drop("SalePrice",axis=1), train_set.SalePrice
X_valid, y_valid = valid_set.drop("SalePrice", axis=1), valid_set.SalePrice

# Check the shape of our X and y variables
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((401125, 54), (401125,), (11573, 54), (11573,))

In [28]:
# creating rsmle function to return the root mean square log error

from sklearn.metrics import mean_squared_log_error, mean_absolute_error

def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))

# Create function to evaluate our model
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_valid, y_valid)}
    return scores

In [27]:
model=RandomForestRegressor(n_jobs=-1,max_samples=30000)
model.fit(X_train,y_train)
show_scores(model)
# model.score(X_train,y_train)

NameError: name 'show_scores' is not defined

***

<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">5.1. PARAMETER TUNING</h4>

**I  have commented below code because tuning doesnt effect on our model performance 
and it takes time to run on my machine**

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import RandomizedSearchCV

# grid_2 = {"n_estimators": [90],
#           "max_depth": [10,15],
#           "min_samples_split": [4,6,8],
#           "min_samples_leaf": [15],
#           "max_features": ["sqrt","auto"],
#           "max_samples": [10000]}

# # create a RandomForestRegressor instance
# rf = RandomForestRegressor(n_jobs=-1,random_state=42)

# # create a RandomizedSearchCV instance
# rs = RandomizedSearchCV(estimator=rf,
#                         param_distributions=grid_2,
#                         n_iter=10,
#                         scoring='neg_mean_squared_error',
#                         cv=5,
#                         random_state=42)

# # fit the RandomizedSearchCV instance to the data
# rs.fit(X_train, y_train)

# # print the best hyperparameters
# print(rs.best_params_)


In [ ]:
# show_scores(rs)

In [29]:
model2=RandomForestRegressor(n_estimators=90,min_samples_split=4,min_samples_leaf=15,max_samples=10000,max_features='auto',
                             max_depth=15)
model2.fit(X_train,y_train)
model2.score(X_train,y_train)

0.8349120627910172

In [30]:
show_scores(model2)

{'Training MAE': 5355.255451819456,
 'Valid MAE': 6270.948236348431,
 'Training RMSLE': 0.2750148824829828,
 'Valid RMSLE': 0.2946849250071983,
 'Training R^2': 0.8349120627910172,
 'Valid R^2': 0.8071113326277606}

***

### Prediction
<h4 style="background-color:#87CEEB;color:white;border-radius: 8px; padding:12px">6.1. prepairing the test data for prediction</h4>

In [31]:
def preprocess_data(df_new):
    # Add datetime parameters for saledate
    df_new["saleyear"] = df_new.saledate.dt.year
    df_new["salemonth"] = df_new.saledate.dt.month
    df_new["saleday"] = df_new.saledate.dt.day

    # Drop saledate
    df_new.drop("saledate", axis=1, inplace=True)
    
    # Fill numeric rows with the median
    for label, content in df_new.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                
                # fill missing values with median
                df_new[label] = content.fillna(content.median())
                
        # Turn categorical variables into numbers
        for label, content in df_new.items():
            if not pd.api.types.is_numeric_dtype(content):
                # Turn categories to numbers +1
                df_new[label] = pd.Categorical(content).codes+1       
    
    return df_new

In [32]:
test_df=preprocess_data(test_data)

In [33]:
test_df.shape

(12457, 54)

In [34]:
print("Test Dataset has",test_df.isnull().values.sum(), "missing values")

Test Dataset has 0 missing values


In [42]:
test_preds = model.predict(test_df)
# Create DataFrame

df_preds = pd.DataFrame()
df_preds["SalesID"] = test_df["SalesID"]
df_preds["SalePrice"] = test_preds


<h4 style="background-color:#90EE90;color:black;border-radius: 8px; padding:12px">2.1.Final solution:</h4>

In [43]:
df_preds

,SalesID,SalePrice
0,1227829,21183.00
1,1227844,16904.50
2,1227847,47855.00
3,1227848,63530.00
4,1227863,46062.50
...,...,...
12452,6643171,47380.00
12453,6643173,12335.00
12454,6643184,12967.50
12455,6643186,16425.00


In [52]:
df_preds.to_csv("test_predition.csv")